<a href="https://colab.research.google.com/github/joeytarnowski/cosc470s24/blob/main/solutions/FastNeuralNetwork-solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The **Hello, world** of neural networks
This code builds a simple neural network that can recognize handwritten 28x28 pixel digits.

Our first version will have 15 neurons in the hidden layer and 10 output neurons.

See image below:
http://neuralnetworksanddeeplearning.com/images/tikz12.png

How many weights in this network?
11760 weights in the first layer (784 * 15) and 150 weights in the second layer (15 * 10) = 11910 weights total
15 biases in the hidden layer and 10 biases in the output layer = 25 biases total.
11910 + 25 = 11935 parameters!

In [3]:
!git clone https://github.com/MichalDanielDobrzanski/DeepLearningPython nn
%cd nn

Cloning into 'nn'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 63 (delta 1), reused 3 (delta 0), pack-reused 55
Receiving objects: 100% (63/63), 16.43 MiB | 19.68 MiB/s, done.
Resolving deltas: 100% (28/28), done.
/content/nn/nn


In [5]:
import network
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
training_data = list(training_data)
test_data = list(test_data)


In [8]:
import numpy as np
print(np.shape(training_data))
print(np.shape(training_data[0][0]))
print(np.shape(training_data[0][1]))
print(training_data[0][1])

(50000, 2)
(784, 1)
(10, 1)
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [27]:
# Solution for Lab 2
# Part 1 - Average digit darkness
# Step 1: Create a function that can calculate the darkness of an individual image
def image_darkness(pixeldata):
  dark_total = 0
  # Adds the value of every pixel into dark_total
  for d in pixeldata:
    dark_total += d
  return dark_total

# Step 2: Now call that function on all the images in the training_data to calculate TEN TOTAL DARKNESSES
# First question - Why will there be 10 darkness totals? - answer this question as a comment in your code.
# ANSWER: There will be 10 darkness totals because there will be one average darkness for each digit 0-9
darkness = image_darkness(training_data[0][0])
whichimage = np.argmax(training_data[0][1])
print(f"The first image was a {whichimage} and its darkness was {darkness}")

# define a 10-element list of darkness totals
darkness_totals = [0 for _ in range(10)]
# define a 10-elment list of digit counts
darkness_counts = [0 for _ in range(10)]

# iterate through each image and update the two data structures
for training_img in training_data:
  darkness = image_darkness(training_img[0])
  whichimage = np.argmax(training_img[1])
  darkness_totals[whichimage] += darkness
  darkness_counts[whichimage] += 1

# Step 3: Now calculate the averages for all ten digits
darkness_averages = [darkness_totals[i]/darkness_counts[i] for i in range(10)]

print(darkness_counts)
print(darkness_averages)



The first image was a 5 and its darkness was [107.51953]
[4932, 5678, 4968, 5101, 4859, 4506, 4951, 5175, 4842, 4988]
[array([135.86891], dtype=float32), array([59.660366], dtype=float32), array([116.29443], dtype=float32), array([110.87087], dtype=float32), array([95.2207], dtype=float32), array([100.346565], dtype=float32), array([107.38318], dtype=float32), array([89.8789], dtype=float32), array([117.880615], dtype=float32), array([96.04564], dtype=float32)]


In [43]:
# Step 4: Use the averages to classify all 10,000 test images.
# Keep track of how many were predicted/classified accurately overall as well as for each individual digit.
# What were the top digit accuracies and which digits?
# ANSWER: The most accurate digits were 0 and 1, which had the highest average darkness and the lowest average darkness, respectively
def find_closest(darkness, darkness_averages):
  # Get the absolute value of darkness minus each darkness average and select the smallest
  counter = 0
  current_closest = (abs(darkness_averages[0] - darkness), counter)
  for digit_avg in darkness_averages:
    if abs(digit_avg - darkness) < current_closest[0]:
      current_closest = (abs(digit_avg - darkness), counter)
    counter += 1
  return current_closest[1]

correct_counts = [0 for _ in range(10)]
wrong_counts = [0 for _ in range(10)]

for imgdata, whichimage in test_data:
  darkness = image_darkness(imgdata)
  # see which darkness_average is closest to "darkness" and make that our classification for this image
  closest_digit = find_closest(darkness, darkness_averages)
  if closest_digit == whichimage:
    correct_counts[whichimage] += 1
  else:
    print(f"\nDarkness: {darkness}")
    print(f"The image incorrectly identified a {whichimage} as a {closest_digit}")
    print(f"Average {whichimage}'s darkness: {darkness_averages[whichimage]}; Average {closest_digit}'s darkness: {darkness_averages[closest_digit]}")
    wrong_counts[whichimage] += 1

# now that we have our correct/wrong counts, turn that into accuracies
digit_accuracies = [correct_counts[i]/(correct_counts[i]+wrong_counts[i]) for i in range(10)]
print(digit_accuracies)

Streaming output truncated to the last 5000 lines.
Darkness: [150.05469]
The image incorrectly identified a 9 as a 0
Average 9's darkness: [96.04564]; Average 0's darkness: [135.86891]

Darkness: [107.21875]
The image incorrectly identified a 0 as a 6
Average 0's darkness: [135.86891]; Average 6's darkness: [107.38318]

Darkness: [165.26172]
The image incorrectly identified a 9 as a 0
Average 9's darkness: [96.04564]; Average 0's darkness: [135.86891]

Darkness: [118.56641]
The image incorrectly identified a 7 as a 8
Average 7's darkness: [89.8789]; Average 8's darkness: [117.880615]

Darkness: [156.6211]
The image incorrectly identified a 6 as a 0
Average 6's darkness: [107.38318]; Average 0's darkness: [135.86891]

Darkness: [101.6875]
The image incorrectly identified a 4 as a 5
Average 4's darkness: [95.2207]; Average 5's darkness: [100.346565]

Darkness: [141.27344]
The image incorrectly identified a 3 as a 0
Average 3's darkness: [110.87087]; Average 0's darkness: [135.86891]

Dar

In [ ]:
list_test_data = list(test_data)
print(np.shape(list_test_data))
print(np.shape(list_test_data[0]))
print(list_test_data[0][1])


(10000, 2)
(2,)
7


In [ ]:
net = network.Network([784, 30, 10])
net.SGD(training_data, 5, 10, 3.0, test_data=test_data)

Epoch 0 complete
Epoch 1 complete
Epoch 2 complete
Epoch 3 complete
Epoch 4 complete


In [ ]:
print(np.shape(net.weights[0]))
print(net.weights[0])

(30, 784)
[[ 0.45745312 -0.43668441  0.06388325 ... -0.01876614  0.69747384
   1.75032045]
 [ 0.31008606 -1.31462574 -0.95998785 ...  2.07556124 -0.23319521
  -0.5521895 ]
 [ 2.62684071  0.59619412  0.75105954 ...  0.39065436  0.21883096
  -0.15008028]
 ...
 [ 0.12553374  1.12652397 -0.19640291 ... -1.06957326 -0.38014393
   2.12662076]
 [-0.33401933 -0.27315002 -0.64691649 ...  0.57483904 -1.0439243
  -0.26700133]
 [-2.64108907  0.69231545 -1.57468679 ... -0.20718688  0.5363602
   0.16273624]]


In [ ]:
print(np.shape(net.weights[1]))
print(net.weights[1])

(10, 30)
[[ 8.92742895e-02 -9.53000886e-01 -4.57105149e-01  8.12327680e-01
  -1.66712716e+00 -3.52723344e+00  2.43366417e+00 -1.17334118e+00
   8.58573096e-01 -6.38617216e-01 -3.98845904e+00  2.60882754e+00
  -3.45033814e+00 -4.68697776e+00 -2.93102636e-02 -2.10035207e+00
  -1.34230789e+00 -6.94064697e-01 -1.19457306e+00 -2.29509311e+00
   3.69451831e+00 -5.47055566e-01  2.43794816e+00 -1.72272752e+00
   2.40744671e-01  1.47444250e+00 -1.05438717e+00 -1.86542911e+00
  -1.06224867e-01 -2.56395561e+00]
 [-7.34977609e-01  4.91942640e+00 -1.66675986e+00  1.59401840e+00
  -2.16068409e+00 -1.51479916e+00 -1.24537721e+00 -1.68016775e+00
   3.37529651e+00 -2.55192130e-01 -6.61492715e-01 -1.81183859e+00
   1.68476200e+00 -3.24190509e-01  1.90014317e+00  1.24494728e+00
  -4.40024242e+00 -1.63853095e+00 -2.36185595e+00 -8.24469577e-01
  -1.22617148e+00 -2.62641360e-01 -3.39598402e+00  1.40744340e+00
  -1.83806995e+00 -7.40020693e-01 -1.30620128e+00  1.84904317e+00
  -2.15747635e+00  7.20940625e-0